# Imports

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import os

# Gather Data

In [2]:
twitter = pd.read_json(r"C:\Programming\Python\Programs_NLP\MLFinalProject\src\data\twitter_data_with_sentiment.json")

In [3]:
twitter.timestamp = pd.to_datetime(twitter.timestamp)

In [7]:
stock_dict = {}
stock_dir = '../src/data/CompleteStockData'

In [12]:
for file in os.listdir('../src/data/CompleteStockData'):
    if file.endswith('.csv'):
        symbol = file.replace('_stocks.csv','')
        stock_dict[symbol] = pd.read_csv(os.sep.join([stock_dir,file]), index_col='date')

In [39]:
for key in stock_dict.keys():
    stock_dict[key].index = pd.to_datetime(stock_dict[key].index)

In [40]:
twitter.head()

,id,text,timestamp,source,symbols,company_names,url,verified,sentiment_compound,sentiment_neg,sentiment_pos,sentiment_neu
0,1019696670777503700,VIDEO: “I was in my office. I was minding my o...,2018-07-18 21:33:26,GoldmanSachs,GS,The Goldman Sachs,https://twitter.com/i/web/status/1019696670777...,True,0.0000,0.000,0.0,1.000
1,1019709091038548000,The price of lumber $LB_F is down 22% since hi...,2018-07-18 22:22:47,StockTwits,M,Macy's,https://twitter.com/i/web/status/1019709091038...,True,0.0000,0.000,0.0,1.000
10,1019723137481617400,U.S. proposes expedited appeal in fight with A...,2018-07-18 23:18:36,Reuters,TWX,Time Warner,https://reut.rs/2O1Ao46,True,-0.3818,0.167,0.0,0.833
100,1016200281749643300,Short sale volume(not short interest) for $D o...,2018-07-09 06:00:01,shortvolumes,AEE,Ameren Corporation,http://shortvolumes.com/?t=D,False,0.0000,0.000,0.0,1.000
1000,1016656436762329100,$MO $SKT $TGT $PNR $ED $T $KMB $BEN $PEP $ORI ...,2018-07-10 12:12:37,SeekingAlpha,ED,Consolidated Edison,https://seekingalpha.com/article/4186312-high-...,False,0.0000,0.000,0.0,1.000


In [41]:
output_df = twitter[twitter.symbols.apply(lambda sym: sym.lower() in stock_dict)][
    ['id', 'timestamp', 'symbols','sentiment_compound', 'sentiment_neg', 'sentiment_pos',
       'sentiment_neu']].copy()

In [49]:
def get_stock_days(sym, ts, output_price='4. close'):
    sym = sym.lower()
    
    before_close = False
    if ts.hour < 16:
        before_close = True
    
    if before_close:
        before_df = stock_dict[sym].loc[:ts + pd.Timedelta(days=1)]
        after_df = stock_dict[sym].loc[ts + pd.Timedelta(days=1):]
    else:
        before_df = stock_dict[sym].loc[:ts ]
        after_df = stock_dict[sym].loc[ts:]
    
    output_df = pd.concat([before_df.iloc[-3:],after_df.iloc[:4]])
    return {'t{}'.format(str(idx)):val for idx,val in enumerate(output_df[output_price], -3)}

In [50]:
get_stock_days('gs', twitter.iloc[0].timestamp)

{'t-3': 231.44,
 't-2': 231.02,
 't-1': 231.24,
 't0': 229.63,
 't1': 231.69,
 't2': 233.76,
 't3': 235.88}

In [51]:
output_df['price_data_dict'] = output_df.apply(lambda row: get_stock_days(row['symbols'], row['timestamp']), axis=1)

In [55]:
for time_slice in ['t{}'.format(str(idx)) for idx in range(-3,4)]:
    output_df[time_slice] = output_df.price_data_dict.apply(lambda d: d.get(time_slice))

In [ ]:
output_df.drop()